### Experiments on Effective Data Augmentation Techniques - Lung Cancer Type Classification Problem



https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images/data


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms

In [10]:
data_path = './data'


In [12]:
transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
    ]),
}